<a href="https://colab.research.google.com/github/forallx94/log_anomaly_detection/blob/main/LDA-for-HDFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## HDFS 1차

In [152]:
!pip install pyldavis

In [153]:
import re
import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from pprint import pprint

In [ ]:
df = pd.read_csv('./data/HDFS_100k.log_structured.csv')

In [ ]:
df

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate
0,1,81109,203518,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>
1,2,81109,203518,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...,E22,BLOCK* NameSystem.allocateBlock:<*>
2,3,81109,203519,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>
3,4,81109,203519,145,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,E5,Receiving block <*> src: /<*> dest: /<*>
4,5,81109,203519,145,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_-1608999687919...,E11,PacketResponder <*> for block <*> terminating
...,...,...,...,...,...,...,...,...,...
104810,104811,81109,210122,13,INFO,dfs.DataBlockScanner,Verification succeeded for blk_-87387097785867...,E2,Verification succeeded for <*>
104811,104812,81109,210124,13,INFO,dfs.DataBlockScanner,Verification succeeded for blk_642047611142564...,E2,Verification succeeded for <*>
104812,104813,81109,210126,13,INFO,dfs.DataBlockScanner,Verification succeeded for blk_-23823897510323...,E2,Verification succeeded for <*>
104813,104814,81109,210126,13,INFO,dfs.DataBlockScanner,Verification succeeded for blk_485603173001003...,E2,Verification succeeded for <*>


In [ ]:
import nltk
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)
text.head()

In [ ]:
data = [i.strip().split(' ') for i in df.Content]

In [ ]:
id2word = corpora.Dictionary(data)

In [ ]:
texts = data

In [ ]:
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]]


In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                            id2word = id2word,
                                            num_topics = 10,
                                            random_state = 100,
                                            update_every =1,
                                            chunksize = 100,
                                            passes = 10,
                                            alpha= 'auto',
                                            per_word_topics = True
                                            )

In [ ]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.046*"BLOCK*" + 0.031*"NameSystem.allocateBlock:" + '
  '0.012*"blk_4241467193520768333" + 0.004*"blk_8909107483987085802" + '
  '0.004*"blk_9047244576407600764" + 0.004*"blk_524408193881860019" + '
  '0.004*"blk_4222802578485615752" + 0.004*"blk_-2951262078924395064" + '
  '0.004*"blk_-9081208173975932165" + 0.004*"blk_8652779608730262978"'),
 (1,
  '0.101*"to" + 0.099*"NameSystem.addStoredBlock:" + 0.099*"size" + 0.099*"is" '
  '+ 0.099*"blockMap" + 0.099*"added" + 0.099*"updated:" + 0.097*"67108864" + '
  '0.095*"BLOCK*" + 0.001*"10.251.39.160:50010"'),
 (2,
  '0.018*"blk_-2607603250149292767" + 0.017*"blk_4286887770593516283" + '
  '0.017*"blk_1037066694877136451" + 0.016*"10.251.74.79:50010" + '
  '0.016*"blk_-7390710706320943439" + 0.016*"blk_-7385019746290967026" + '
  '0.016*"blk_-1160765867676592212" + 0.015*"10.251.31.5:50010" + '
  '0.015*"blk_-4130422002907235753" + 0.015*"blk_-372632911736985200"'),
 (3,
  '0.157*"block" + 0.151*"of" + 0.150*"from" + 0.150*"size" 

In [ ]:
import pyLDAvis
import pyLDAvis.gensim as gensimvis

In [ ]:
# # 시각화에 1시간 이상 소모
# pyLDAvis.enable_notebook()
# vis = gensimvis.prepare(lda_model, corpus, id2word)
# pyLDAvis.display(vis)

KeyboardInterrupt: ignored

## HDFS 2차

* https://omicro03.medium.com/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC-nlp-8%EC%9D%BC%EC%B0%A8-lda-f571b4da9d04

In [ ]:
text = df[['LineId','Content']]
text.head()

,LineId,Content
0,1,Receiving block blk_-1608999687919862906 src: ...
1,2,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...
2,3,Receiving block blk_-1608999687919862906 src: ...
3,4,Receiving block blk_-1608999687919862906 src: ...
4,5,PacketResponder 1 for block blk_-1608999687919...


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# 토큰화
text['Content'] = text.apply(lambda row: nltk.word_tokenize(row['Content']), axis=1)
text.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,LineId,Content
0,1,"[Receiving, block, blk_-1608999687919862906, s..."
1,2,"[BLOCK*, NameSystem.allocateBlock, :, /mnt/had..."
2,3,"[Receiving, block, blk_-1608999687919862906, s..."
3,4,"[Receiving, block, blk_-1608999687919862906, s..."
4,5,"[PacketResponder, 1, for, block, blk_-16089996..."


In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# 불용어 처리
stop = stopwords.words('english')
text['Content'] = text['Content'].apply(lambda x: [word for word in x if word not in (stop)])
text.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,LineId,Content
0,1,"[Receiving, block, blk_-1608999687919862906, s..."
1,2,"[BLOCK*, NameSystem.allocateBlock, :, /mnt/had..."
2,3,"[Receiving, block, blk_-1608999687919862906, s..."
3,4,"[Receiving, block, blk_-1608999687919862906, s..."
4,5,"[PacketResponder, 1, block, blk_-1608999687919..."


In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# 표제어 추출
from nltk.stem import WordNetLemmatizer
text['Content'] = text['Content'].apply(lambda x : [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])
text.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,LineId,Content
0,1,"[Receiving, block, blk_-1608999687919862906, s..."
1,2,"[BLOCK*, NameSystem.allocateBlock, :, /mnt/had..."
2,3,"[Receiving, block, blk_-1608999687919862906, s..."
3,4,"[Receiving, block, blk_-1608999687919862906, s..."
4,5,"[PacketResponder, 1, block, blk_-1608999687919..."


In [ ]:
# 일정 길이 이하 단어 삭제 
tokenized_doc = text['Content'].apply(lambda x : [word for word in x if len(word) > 3])
tokenized_doc[:5]

0    [Receiving, block, blk_-1608999687919862906, /...
1    [BLOCK*, NameSystem.allocateBlock, /mnt/hadoop...
2    [Receiving, block, blk_-1608999687919862906, /...
3    [Receiving, block, blk_-1608999687919862906, /...
4    [PacketResponder, block, blk_-1608999687919862...
Name: Content, dtype: object

In [ ]:
# 역토큰화
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
text['Content'] = detokenized_doc
# 다시 text['Content'] 에 저장
text.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,LineId,Content
0,1,Receiving block blk_-1608999687919862906 /10.2...
1,2,BLOCK* NameSystem.allocateBlock /mnt/hadoop/ma...
2,3,Receiving block blk_-1608999687919862906 /10.2...
3,4,Receiving block blk_-1608999687919862906 /10.2...
4,5,PacketResponder block blk_-1608999687919862906...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(text['Content'])
X.shape

(104815, 1000)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', random_state=777, max_iter=1)
lda_top = lda_model.fit_transform(X)
print(lda_model.components_)
print(lda_model.components_.shape)

[[1.00000006e-01 1.00000006e-01 1.00000008e-01 ... 1.00000042e-01
  1.00000818e-01 1.00000007e-01]
 [1.89001672e+00 4.28374804e+00 2.82443277e+00 ... 2.00300039e+03
  1.00000107e-01 1.00000008e-01]
 [1.00000006e-01 1.00000009e-01 1.00000007e-01 ... 1.00000007e-01
  1.00000014e-01 1.00000007e-01]
 ...
 [1.00000008e-01 1.00000007e-01 1.00000007e-01 ... 1.00000008e-01
  1.00000403e-01 1.00000007e-01]
 [1.00017155e-01 1.00000006e-01 1.00000008e-01 ... 1.00000032e-01
  6.04580620e+03 1.00000007e-01]
 [1.00000006e-01 1.00000008e-01 1.00000007e-01 ... 1.00000122e-01
  1.00000184e-01 1.00000007e-01]]
(10, 1000)


In [ ]:
terms = vectorizer.get_feature_names() 
# 단어 집합. 1,000개의 단어가 저장되어있음.
def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d :" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n -1:-1]])
get_topics(lda_model.components_, terms)

Topic 1 : [('107', 1102.18), ('251', 1081.04), ('30', 1007.07), ('10', 951.24), ('134', 779.46)]
Topic 2 : [('root', 2003.0), ('user', 2003.0), ('rand', 2003.0), ('allocateblock', 2002.79), ('_temporary', 2002.77)]
Topic 3 : [('addstoredblock', 6126.61), ('blockmap', 6124.14), ('update', 6124.14), ('namesystem', 5393.07), ('50010', 3903.47)]
Topic 4 : [('10', 1478.77), ('250', 1312.99), ('received', 1171.09), ('size', 1045.39), ('67108864', 969.62)]
Topic 5 : [('251', 1343.46), ('10', 1232.41), ('203', 1066.3), ('received', 1034.8), ('123', 994.32)]
Topic 6 : [('70', 1141.97), ('194', 1058.49), ('251', 869.47), ('67', 792.84), ('10', 749.25)]
Topic 7 : [('214', 1701.53), ('251', 1019.8), ('10', 978.46), ('received', 834.1), ('74', 778.95)]
Topic 8 : [('251', 1207.42), ('10', 1055.27), ('31', 1029.24), ('111', 1027.74), ('192', 923.02)]
Topic 9 : [('packetresponder', 15604.81), ('terminate', 15602.6), ('block', 6559.31), ('blk_', 6069.46), ('verification', 6045.81)]
Topic 10 : [('10', 1

## HDFS 3차
1차 + 2차

In [ ]:
text = df[['LineId','Content']]
text.head()

,LineId,Content
0,1,Receiving block blk_-1608999687919862906 src: ...
1,2,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...
2,3,Receiving block blk_-1608999687919862906 src: ...
3,4,Receiving block blk_-1608999687919862906 src: ...
4,5,PacketResponder 1 for block blk_-1608999687919...


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# 토큰화
text['Content'] = text.apply(lambda row: nltk.word_tokenize(row['Content']), axis=1)
text.head()

,LineId,Content
0,1,"[Receiving, block, blk_-1608999687919862906, s..."
1,2,"[BLOCK*, NameSystem.allocateBlock, :, /mnt/had..."
2,3,"[Receiving, block, blk_-1608999687919862906, s..."
3,4,"[Receiving, block, blk_-1608999687919862906, s..."
4,5,"[PacketResponder, 1, for, block, blk_-16089996..."


In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# 불용어 처리
stop = stopwords.words('english')
text['Content'] = text['Content'].apply(lambda x: [word for word in x if word not in (stop)])
text.head()

,LineId,Content
0,1,"[Receiving, block, blk_-1608999687919862906, s..."
1,2,"[BLOCK*, NameSystem.allocateBlock, :, /mnt/had..."
2,3,"[Receiving, block, blk_-1608999687919862906, s..."
3,4,"[Receiving, block, blk_-1608999687919862906, s..."
4,5,"[PacketResponder, 1, block, blk_-1608999687919..."


In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# 표제어 추출
from nltk.stem import WordNetLemmatizer
text['Content'] = text['Content'].apply(lambda x : [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])
text.head()

,LineId,Content
0,1,"[Receiving, block, blk_-1608999687919862906, s..."
1,2,"[BLOCK*, NameSystem.allocateBlock, :, /mnt/had..."
2,3,"[Receiving, block, blk_-1608999687919862906, s..."
3,4,"[Receiving, block, blk_-1608999687919862906, s..."
4,5,"[PacketResponder, 1, block, blk_-1608999687919..."


In [ ]:
# 일정 길이 이하 단어 삭제 
tokenized_doc = text['Content'].apply(lambda x : [word for word in x if len(word) > 3])
tokenized_doc[:5]

0    [Receiving, block, blk_-1608999687919862906, /...
1    [BLOCK*, NameSystem.allocateBlock, /mnt/hadoop...
2    [Receiving, block, blk_-1608999687919862906, /...
3    [Receiving, block, blk_-1608999687919862906, /...
4    [PacketResponder, block, blk_-1608999687919862...
Name: Content, dtype: object

In [ ]:
data = [i for i in tokenized_doc]

In [ ]:
id2word = corpora.Dictionary(data)

In [ ]:
corpus = [id2word.doc2bow(text) for text in data]

In [ ]:
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [ ]:
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                            id2word = id2word,
                                            num_topics = 10,
                                            random_state = 100,
                                            update_every =1,
                                            chunksize = 100,
                                            passes = 10,
                                            alpha= 'auto',
                                            per_word_topics = True
                                            )

In [ ]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.307*"block" + 0.015*"blk_-3059453549958878582" + 0.013*"/10.251.31.5" + '
  '0.012*"/10.251.126.5" + 0.011*"/10.250.7.32" + 0.011*"/10.251.203.129" + '
  '0.011*"/10.250.11.85" + 0.010*"/10.251.71.97" + '
  '0.010*"blk_-836233151529212612" + 0.009*"/10.251.39.192"'),
 (1,
  '0.234*"block" + 0.016*"blk_-2607603250149292767" + '
  '0.014*"blk_3126100339057783399" + 0.014*"10.251.74.79:50010" + '
  '0.014*"blk_-598955920782630733" + 0.013*"blk_4718466536212741328" + '
  '0.012*"/10.251.111.209" + 0.012*"10.251.111.209:50010" + '
  '0.011*"/10.251.111.37" + 0.011*"blk_7907797724760003565"'),
 (2,
  '0.200*"size" + 0.195*"67108864" + 0.101*"Received" + '
  '0.100*"NameSystem.addStoredBlock" + 0.100*"update" + 0.100*"blockMap" + '
  '0.091*"BLOCK*" + 0.001*"10.251.39.160:50010" + 0.001*"10.251.39.209:50010" '
  '+ 0.001*"10.251.107.227:50010"'),
 (3,
  '0.060*"BLOCK*" + 0.028*"NameSystem.allocateBlock" + '
  '0.004*"blk_8909107483987085802" + 0.004*"blk_9047244576407600764" + '
  '

## HDFS 1차 3차 단어 정리

1차 topic

In [ ]:
_1st = [(0,
  '0.046*"BLOCK*" + 0.031*"NameSystem.allocateBlock:" + '
  '0.012*"blk_4241467193520768333" + 0.004*"blk_8909107483987085802" + '
  '0.004*"blk_9047244576407600764" + 0.004*"blk_524408193881860019" + '
  '0.004*"blk_4222802578485615752" + 0.004*"blk_-2951262078924395064" + '
  '0.004*"blk_-9081208173975932165" + 0.004*"blk_8652779608730262978"'),
 (1,
  '0.101*"to" + 0.099*"NameSystem.addStoredBlock:" + 0.099*"size" + 0.099*"is" '
  '+ 0.099*"blockMap" + 0.099*"added" + 0.099*"updated:" + 0.097*"67108864" + '
  '0.095*"BLOCK*" + 0.001*"10.251.39.160:50010"'),
 (2,
  '0.018*"blk_-2607603250149292767" + 0.017*"blk_4286887770593516283" + '
  '0.017*"blk_1037066694877136451" + 0.016*"10.251.74.79:50010" + '
  '0.016*"blk_-7390710706320943439" + 0.016*"blk_-7385019746290967026" + '
  '0.016*"blk_-1160765867676592212" + 0.015*"10.251.31.5:50010" + '
  '0.015*"blk_-4130422002907235753" + 0.015*"blk_-372632911736985200"'),
 (3,
  '0.157*"block" + 0.151*"of" + 0.150*"from" + 0.150*"size" + 0.150*"Received" '
  '+ 0.146*"67108864" + 0.050*"1" + 0.001*"/10.251.31.5" + '
  '0.001*"/10.250.7.32" + 0.001*"/10.251.203.129"'),
 (4,
  '0.159*"block" + 0.145*"src:" + 0.145*"dest:" + 0.144*"Receiving" + '
  '0.005*"blk_4108032757710290196" + 0.005*"blk_-7326879742777330616" + '
  '0.005*"blk_4966276028193818154" + 0.005*"blk_-955678844525403431" + '
  '0.005*"blk_-4566764090560972175" + 0.005*"blk_7359588822015996941"'),
 (5,
  '0.014*"blk_-7199733435216834133" + 0.013*"blk_-3140754468249228022" + '
  '0.012*"blk_-8608758988186237206" + 0.012*"blk_-7374177486939901806" + '
  '0.012*"blk_7007568649123695530" + 0.011*"blk_252816358434654494" + '
  '0.011*"/10.251.126.5" + 0.011*"blk_4301761665496587132" + '
  '0.011*"/10.251.31.160" + 0.011*"/10.251.194.245"'),
 (6,
  '0.013*"/10.251.39.209" + 0.012*"/10.251.107.227" + '
  '0.011*"blk_-365681253977708548" + 0.011*"/10.251.203.179" + '
  '0.010*"/10.251.26.131" + 0.010*"/10.251.111.209" + 0.010*"/10.251.105.189" '
  '+ 0.009*"10.251.111.209:50010" + 0.009*"/10.251.43.21" + '
  '0.009*"/10.250.17.225"'),
 (7,
  '0.386*"for" + 0.157*"PacketResponder" + 0.156*"terminating" + 0.147*"block" '
  '+ 0.052*"0" + 0.052*"2" + 0.001*"/10.251.71.16" + 0.001*"/10.250.11.100" + '
  '0.001*"/10.251.197.226" + 0.001*"/10.250.14.196"'),
 (8,
  '0.320*"succeeded" + 0.320*"Verification" + 0.002*"blk_6420476111425645508" '
  '+ 0.002*"blk_4747238186020862016" + 0.002*"blk_-2382389751032389929" + '
  '0.002*"blk_7455154075126513420" + 0.002*"blk_4856031730010032819" + '
  '0.002*"blk_-3155415637543545820" + 0.002*"blk_638939784120214749" + '
  '0.002*"blk_-2947515393396507971"'),
 (9,
  '0.018*"blk_-9039789976760025618" + 0.018*"blk_8232653167951816215" + '
  '0.018*"blk_-3059453549958878582" + 0.017*"blk_9010689441969286891" + '
  '0.016*"blk_-1214394198778192417" + 0.015*"blk_-4358861442329520889" + '
  '0.015*"blk_-271233912558262172" + 0.014*"blk_5251082346485479848" + '
  '0.014*"blk_4472097162321285264" + 0.014*"blk_-4917509153767079107"')]

In [ ]:
result_df_1 = pd.DataFrame([i[1].split(' + ') for i in _1st])

In [ ]:
for i in result_df_1:
  result_df_1['{}_rate'.format(i)]=result_df_1[i].str[:5]
  result_df_1[i]= result_df_1[i].str[6:]

In [ ]:
temp = []
for i in range(10):
  temp += ['{}_rate'.format(i), i]

In [ ]:
result_df_1 = result_df_1[temp]

3차 topic

In [ ]:
_3rd = [(0,
  '0.307*"block" + 0.015*"blk_-3059453549958878582" + 0.013*"/10.251.31.5" + '
  '0.012*"/10.251.126.5" + 0.011*"/10.250.7.32" + 0.011*"/10.251.203.129" + '
  '0.011*"/10.250.11.85" + 0.010*"/10.251.71.97" + '
  '0.010*"blk_-836233151529212612" + 0.009*"/10.251.39.192"'),
 (1,
  '0.234*"block" + 0.016*"blk_-2607603250149292767" + '
  '0.014*"blk_3126100339057783399" + 0.014*"10.251.74.79:50010" + '
  '0.014*"blk_-598955920782630733" + 0.013*"blk_4718466536212741328" + '
  '0.012*"/10.251.111.209" + 0.012*"10.251.111.209:50010" + '
  '0.011*"/10.251.111.37" + 0.011*"blk_7907797724760003565"'),
 (2,
  '0.200*"size" + 0.195*"67108864" + 0.101*"Received" + '
  '0.100*"NameSystem.addStoredBlock" + 0.100*"update" + 0.100*"blockMap" + '
  '0.091*"BLOCK*" + 0.001*"10.251.39.160:50010" + 0.001*"10.251.39.209:50010" '
  '+ 0.001*"10.251.107.227:50010"'),
 (3,
  '0.060*"BLOCK*" + 0.028*"NameSystem.allocateBlock" + '
  '0.004*"blk_8909107483987085802" + 0.004*"blk_9047244576407600764" + '
  '0.004*"blk_524408193881860019" + 0.004*"blk_4222802578485615752" + '
  '0.004*"blk_-2951262078924395064" + 0.004*"blk_-9081208173975932165" + '
  '0.004*"blk_8652779608730262978" + 0.004*"blk_-976339114197076853"'),
 (4,
  '0.318*"block" + 0.307*"PacketResponder" + 0.306*"terminate" + '
  '0.002*"/10.251.30.179" + 0.002*"10.251.90.64:50010" + '
  '0.001*"/10.250.11.100" + 0.001*"blk_-7722134978135978544" + '
  '0.001*"blk_5951221751432246380" + 0.001*"10.250.7.244:50010" + '
  '0.001*"blk_-1196019860597610288"'),
 (5,
  '0.020*"blk_-6742986629304373462" + 0.019*"blk_-5139777489517063680" + '
  '0.019*"blk_2907885683673342872" + 0.018*"blk_-3260736406913425394" + '
  '0.017*"10.251.106.10:50010" + 0.017*"blk_-861765373330722953" + '
  '0.016*"blk_-1402764052044494020" + 0.016*"blk_-3350625939026579183" + '
  '0.016*"blk_-7800853382581848688" + 0.013*"blk_4509878823587110787"'),
 (6,
  '0.265*"succeed" + 0.265*"Verification" + 0.097*"block" + '
  '0.002*"blk_-2382389751032389929" + 0.002*"blk_4747238186020862016" + '
  '0.002*"blk_6420476111425645508" + 0.002*"/10.251.39.209" + '
  '0.002*"blk_4856031730010032819" + 0.002*"blk_-3155415637543545820" + '
  '0.002*"/10.251.107.227"'),
 (7,
  '0.057*"blk_4241467193520768333" + 0.016*"blk_1037066694877136451" + '
  '0.016*"blk_-7325968241997525720" + 0.015*"blk_-7390710706320943439" + '
  '0.015*"blk_-1214394198778192417" + 0.015*"blk_-7374177486939901806" + '
  '0.014*"blk_-4358861442329520889" + 0.014*"blk_-372632911736985200" + '
  '0.013*"blk_-1322963786995945056" + 0.013*"blk_-7858897319223152664"'),
 (8,
  '0.192*"block" + 0.190*"dest" + 0.189*"Receiving" + '
  '0.007*"blk_4966276028193818154" + 0.007*"blk_4108032757710290196" + '
  '0.007*"blk_-7326879742777330616" + 0.006*"blk_-955678844525403431" + '
  '0.006*"blk_-4566764090560972175" + 0.006*"blk_8093107867996361765" + '
  '0.005*"blk_6157997286876178234"'),
 (9,
  '0.169*"block" + 0.017*"blk_4286887770593516283" + '
  '0.016*"blk_1207568959074804298" + 0.016*"blk_-8608758988186237206" + '
  '0.015*"10.251.31.5:50010" + 0.015*"blk_1200401807679484048" + '
  '0.014*"/10.251.194.245" + 0.013*"/10.251.105.189" + '
  '0.013*"blk_-6174155369373380263" + 0.012*"/10.251.126.227"')]

In [ ]:
result_df_2 = pd.DataFrame([i[1].split(' + ') for i in _3rd])

In [ ]:
for i in result_df_2:
  result_df_2['{}_rate'.format(i)]=result_df_2[i].str[:5]
  result_df_2[i]= result_df_2[i].str[6:]

In [ ]:
temp = []
for i in range(10):
  temp += ['{}_rate'.format(i), i]

In [ ]:
result_df_2 = result_df_2[temp]

결과 확인

In [ ]:
result_df_1

,0_rate,0,1_rate,1,2_rate,2,3_rate,3,4_rate,4,5_rate,5,6_rate,6,7_rate,7,8_rate,8,9_rate,9
0,0.046,"""BLOCK*""",0.031,"""NameSystem.allocateBlock:""",0.012,"""blk_4241467193520768333""",0.004,"""blk_8909107483987085802""",0.004,"""blk_9047244576407600764""",0.004,"""blk_524408193881860019""",0.004,"""blk_4222802578485615752""",0.004,"""blk_-2951262078924395064""",0.004,"""blk_-9081208173975932165""",0.004,"""blk_8652779608730262978"""
1,0.101,"""to""",0.099,"""NameSystem.addStoredBlock:""",0.099,"""size""",0.099,"""is""",0.099,"""blockMap""",0.099,"""added""",0.099,"""updated:""",0.097,"""67108864""",0.095,"""BLOCK*""",0.001,"""10.251.39.160:50010"""
2,0.018,"""blk_-2607603250149292767""",0.017,"""blk_4286887770593516283""",0.017,"""blk_1037066694877136451""",0.016,"""10.251.74.79:50010""",0.016,"""blk_-7390710706320943439""",0.016,"""blk_-7385019746290967026""",0.016,"""blk_-1160765867676592212""",0.015,"""10.251.31.5:50010""",0.015,"""blk_-4130422002907235753""",0.015,"""blk_-372632911736985200"""
3,0.157,"""block""",0.151,"""of""",0.150,"""from""",0.150,"""size""",0.150,"""Received""",0.146,"""67108864""",0.050,"""1""",0.001,"""/10.251.31.5""",0.001,"""/10.250.7.32""",0.001,"""/10.251.203.129"""
4,0.159,"""block""",0.145,"""src:""",0.145,"""dest:""",0.144,"""Receiving""",0.005,"""blk_4108032757710290196""",0.005,"""blk_-7326879742777330616""",0.005,"""blk_4966276028193818154""",0.005,"""blk_-955678844525403431""",0.005,"""blk_-4566764090560972175""",0.005,"""blk_7359588822015996941"""
5,0.014,"""blk_-7199733435216834133""",0.013,"""blk_-3140754468249228022""",0.012,"""blk_-8608758988186237206""",0.012,"""blk_-7374177486939901806""",0.012,"""blk_7007568649123695530""",0.011,"""blk_252816358434654494""",0.011,"""/10.251.126.5""",0.011,"""blk_4301761665496587132""",0.011,"""/10.251.31.160""",0.011,"""/10.251.194.245"""
6,0.013,"""/10.251.39.209""",0.012,"""/10.251.107.227""",0.011,"""blk_-365681253977708548""",0.011,"""/10.251.203.179""",0.010,"""/10.251.26.131""",0.010,"""/10.251.111.209""",0.010,"""/10.251.105.189""",0.009,"""10.251.111.209:50010""",0.009,"""/10.251.43.21""",0.009,"""/10.250.17.225"""
7,0.386,"""for""",0.157,"""PacketResponder""",0.156,"""terminating""",0.147,"""block""",0.052,"""0""",0.052,"""2""",0.001,"""/10.251.71.16""",0.001,"""/10.250.11.100""",0.001,"""/10.251.197.226""",0.001,"""/10.250.14.196"""
8,0.320,"""succeeded""",0.320,"""Verification""",0.002,"""blk_6420476111425645508""",0.002,"""blk_4747238186020862016""",0.002,"""blk_-2382389751032389929""",0.002,"""blk_7455154075126513420""",0.002,"""blk_4856031730010032819""",0.002,"""blk_-3155415637543545820""",0.002,"""blk_638939784120214749""",0.002,"""blk_-2947515393396507971"""
9,0.018,"""blk_-9039789976760025618""",0.018,"""blk_8232653167951816215""",0.018,"""blk_-3059453549958878582""",0.017,"""blk_9010689441969286891""",0.016,"""blk_-1214394198778192417""",0.015,"""blk_-4358861442329520889""",0.015,"""blk_-271233912558262172""",0.014,"""blk_5251082346485479848""",0.014,"""blk_4472097162321285264""",0.014,"""blk_-4917509153767079107"""


In [ ]:
result_df_2

,0_rate,0,1_rate,1,2_rate,2,3_rate,3,4_rate,4,5_rate,5,6_rate,6,7_rate,7,8_rate,8,9_rate,9
0,0.307,"""block""",0.015,"""blk_-3059453549958878582""",0.013,"""/10.251.31.5""",0.012,"""/10.251.126.5""",0.011,"""/10.250.7.32""",0.011,"""/10.251.203.129""",0.011,"""/10.250.11.85""",0.010,"""/10.251.71.97""",0.010,"""blk_-836233151529212612""",0.009,"""/10.251.39.192"""
1,0.234,"""block""",0.016,"""blk_-2607603250149292767""",0.014,"""blk_3126100339057783399""",0.014,"""10.251.74.79:50010""",0.014,"""blk_-598955920782630733""",0.013,"""blk_4718466536212741328""",0.012,"""/10.251.111.209""",0.012,"""10.251.111.209:50010""",0.011,"""/10.251.111.37""",0.011,"""blk_7907797724760003565"""
2,0.200,"""size""",0.195,"""67108864""",0.101,"""Received""",0.100,"""NameSystem.addStoredBlock""",0.100,"""update""",0.100,"""blockMap""",0.091,"""BLOCK*""",0.001,"""10.251.39.160:50010""",0.001,"""10.251.39.209:50010""",0.001,"""10.251.107.227:50010"""
3,0.060,"""BLOCK*""",0.028,"""NameSystem.allocateBlock""",0.004,"""blk_8909107483987085802""",0.004,"""blk_9047244576407600764""",0.004,"""blk_524408193881860019""",0.004,"""blk_4222802578485615752""",0.004,"""blk_-2951262078924395064""",0.004,"""blk_-9081208173975932165""",0.004,"""blk_8652779608730262978""",0.004,"""blk_-976339114197076853"""
4,0.318,"""block""",0.307,"""PacketResponder""",0.306,"""terminate""",0.002,"""/10.251.30.179""",0.002,"""10.251.90.64:50010""",0.001,"""/10.250.11.100""",0.001,"""blk_-7722134978135978544""",0.001,"""blk_5951221751432246380""",0.001,"""10.250.7.244:50010""",0.001,"""blk_-1196019860597610288"""
5,0.020,"""blk_-6742986629304373462""",0.019,"""blk_-5139777489517063680""",0.019,"""blk_2907885683673342872""",0.018,"""blk_-3260736406913425394""",0.017,"""10.251.106.10:50010""",0.017,"""blk_-861765373330722953""",0.016,"""blk_-1402764052044494020""",0.016,"""blk_-3350625939026579183""",0.016,"""blk_-7800853382581848688""",0.013,"""blk_4509878823587110787"""
6,0.265,"""succeed""",0.265,"""Verification""",0.097,"""block""",0.002,"""blk_-2382389751032389929""",0.002,"""blk_4747238186020862016""",0.002,"""blk_6420476111425645508""",0.002,"""/10.251.39.209""",0.002,"""blk_4856031730010032819""",0.002,"""blk_-3155415637543545820""",0.002,"""/10.251.107.227"""
7,0.057,"""blk_4241467193520768333""",0.016,"""blk_1037066694877136451""",0.016,"""blk_-7325968241997525720""",0.015,"""blk_-7390710706320943439""",0.015,"""blk_-1214394198778192417""",0.015,"""blk_-7374177486939901806""",0.014,"""blk_-4358861442329520889""",0.014,"""blk_-372632911736985200""",0.013,"""blk_-1322963786995945056""",0.013,"""blk_-7858897319223152664"""
8,0.192,"""block""",0.190,"""dest""",0.189,"""Receiving""",0.007,"""blk_4966276028193818154""",0.007,"""blk_4108032757710290196""",0.007,"""blk_-7326879742777330616""",0.006,"""blk_-955678844525403431""",0.006,"""blk_-4566764090560972175""",0.006,"""blk_8093107867996361765""",0.005,"""blk_6157997286876178234"""
9,0.169,"""block""",0.017,"""blk_4286887770593516283""",0.016,"""blk_1207568959074804298""",0.016,"""blk_-8608758988186237206""",0.015,"""10.251.31.5:50010""",0.015,"""blk_1200401807679484048""",0.014,"""/10.251.194.245""",0.013,"""/10.251.105.189""",0.013,"""blk_-6174155369373380263""",0.012,"""/10.251.126.227"""


## Syslog 1차

In [157]:
df = pd.read_excel('./data/syslog 샘플.xls')
df.head()

,심각도,발생 일시,그룹경로,시스템명,호스트명,리소스 이름,대상,이벤트 타입,이벤트 소스,내용
0,ERROR,2021-01-08 11:39:07,인프라그룹,Syslog Receiver,NaN,Syslog Receiver,인프라그룹>Syslog Receiver,syslog,172.30.151.88,[syslog]<43>Jan 8 11:39:07 Kolon_L7_trust_02 ...
1,ERROR,2021-01-08 11:39:07,인프라그룹,Syslog Receiver,NaN,Syslog Receiver,인프라그룹>Syslog Receiver,syslog,172.30.151.88,[syslog]<43>Jan 8 11:39:07 Kolon_L7_trust_02 ...
2,ERROR,2021-01-08 11:39:06,인프라그룹,Syslog Receiver,NaN,Syslog Receiver,인프라그룹>Syslog Receiver,syslog,172.30.151.87,[syslog]<43>Jan 8 11:39:06 Kolon_L7_trust_01 ...
3,ERROR,2021-01-08 11:38:07,인프라그룹,Syslog Receiver,NaN,Syslog Receiver,인프라그룹>Syslog Receiver,syslog,172.30.151.88,[syslog]<43>Jan 8 11:38:07 Kolon_L7_trust_02 ...
4,ERROR,2021-01-08 11:38:07,인프라그룹,Syslog Receiver,NaN,Syslog Receiver,인프라그룹>Syslog Receiver,syslog,172.30.151.88,[syslog]<43>Jan 8 11:38:07 Kolon_L7_trust_02 ...


In [161]:
text = df[['내용']]
text.columns = ['Content']
text.head()

,Content
0,[syslog]<43>Jan 8 11:39:07 Kolon_L7_trust_02 ...
1,[syslog]<43>Jan 8 11:39:07 Kolon_L7_trust_02 ...
2,[syslog]<43>Jan 8 11:39:06 Kolon_L7_trust_01 ...
3,[syslog]<43>Jan 8 11:38:07 Kolon_L7_trust_02 ...
4,[syslog]<43>Jan 8 11:38:07 Kolon_L7_trust_02 ...


In [162]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [163]:
# 토큰화
text['Content'] = text.apply(lambda row: nltk.word_tokenize(row['Content']), axis=1)
text.head()

,Content
0,"[[, syslog, ], <, 43, >, Jan, 8, 11:39:07, Kol..."
1,"[[, syslog, ], <, 43, >, Jan, 8, 11:39:07, Kol..."
2,"[[, syslog, ], <, 43, >, Jan, 8, 11:39:06, Kol..."
3,"[[, syslog, ], <, 43, >, Jan, 8, 11:38:07, Kol..."
4,"[[, syslog, ], <, 43, >, Jan, 8, 11:38:07, Kol..."


In [164]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [165]:
# 불용어 처리
stop = stopwords.words('english')
text['Content'] = text['Content'].apply(lambda x: [word for word in x if word not in (stop)])
text.head()

,Content
0,"[[, syslog, ], <, 43, >, Jan, 8, 11:39:07, Kol..."
1,"[[, syslog, ], <, 43, >, Jan, 8, 11:39:07, Kol..."
2,"[[, syslog, ], <, 43, >, Jan, 8, 11:39:06, Kol..."
3,"[[, syslog, ], <, 43, >, Jan, 8, 11:38:07, Kol..."
4,"[[, syslog, ], <, 43, >, Jan, 8, 11:38:07, Kol..."


In [166]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [167]:
# 표제어 추출
from nltk.stem import WordNetLemmatizer
text['Content'] = text['Content'].apply(lambda x : [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])
text.head()

,Content
0,"[[, syslog, ], <, 43, >, Jan, 8, 11:39:07, Kol..."
1,"[[, syslog, ], <, 43, >, Jan, 8, 11:39:07, Kol..."
2,"[[, syslog, ], <, 43, >, Jan, 8, 11:39:06, Kol..."
3,"[[, syslog, ], <, 43, >, Jan, 8, 11:38:07, Kol..."
4,"[[, syslog, ], <, 43, >, Jan, 8, 11:38:07, Kol..."


In [168]:
# 일정 길이 이하 단어 삭제 
tokenized_doc = text['Content'].apply(lambda x : [word for word in x if len(word) > 3])
tokenized_doc[:5]

0    [syslog, 11:39:07, Kolon_L7_trust_02, syslog-n...
1    [syslog, 11:39:07, Kolon_L7_trust_02, syslog-n...
2    [syslog, 11:39:06, Kolon_L7_trust_01, syslog-n...
3    [syslog, 11:38:07, Kolon_L7_trust_02, syslog-n...
4    [syslog, 11:38:07, Kolon_L7_trust_02, syslog-n...
Name: Content, dtype: object

In [169]:
data = [i for i in tokenized_doc]

In [174]:
id2word = corpora.Dictionary(data)

In [175]:
corpus = [id2word.doc2bow(text) for text in data]

In [176]:
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]]


In [177]:
import warnings 
warnings.filterwarnings('ignore')

In [178]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                            id2word = id2word,
                                            num_topics = 10,
                                            random_state = 100,
                                            update_every =1,
                                            chunksize = 100,
                                            passes = 10,
                                            alpha= 'auto',
                                            per_word_topics = True
                                            )

In [179]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.022*"BMW_Opo_AS" + 0.022*"ge-1/0/0.0" + 0.022*"192.168.0.37" + '
  '0.010*"779b9feb" + 0.010*"2021-01-06T18:39:28.539+09:00" + 0.010*"e7e5" + '
  '0.003*"08:15:06" + 0.003*"03:32:08" + 0.003*"03:40:08" + 0.003*"03:47:08"'),
 (1,
  '0.084*"syslog-ng" + 0.084*"syslog" + 0.083*"Kolon_L7_trust_01" + '
  '0.083*"12592" + 0.083*"fd=\'39" + 0.083*"occur" + 0.083*"write" + '
  '0.083*"error=\'Connection" + 0.083*"refuse" + 0.083*"error"'),
 (2,
  '0.109*"idle" + 0.109*"POLLERR" + 0.109*"occur" + 0.109*"fd=\'39" + '
  '0.109*"syslog-ng" + 0.109*"syslog" + 0.109*"Kolon_L7_trust_01" + '
  '0.109*"12592" + 0.001*"18:51:11" + 0.001*"18:46:10"'),
 (3,
  '0.123*"syslog" + 0.123*"syslog-ng" + 0.123*"Kolon_L7_trust_01" + '
  '0.123*"12592" + 0.122*"time_reopen=\'60" + 0.122*"Connection" + '
  '0.122*"break" + 0.001*"18:55:11" + 0.000*"18:51:11" + 0.000*"18:49:10"'),
 (4,
  '0.023*"iken" + 0.014*"members" + 0.014*"available" + '
  '0.014*"/Common/p_media_9005" + 0.014*"local0" + 0.008*"0101022

결과 정리

In [180]:
_1st = [(0,
  '0.022*"BMW_Opo_AS" + 0.022*"ge-1/0/0.0" + 0.022*"192.168.0.37" + '
  '0.010*"779b9feb" + 0.010*"2021-01-06T18:39:28.539+09:00" + 0.010*"e7e5" + '
  '0.003*"08:15:06" + 0.003*"03:32:08" + 0.003*"03:40:08" + 0.003*"03:47:08"'),
 (1,
  '0.084*"syslog-ng" + 0.084*"syslog" + 0.083*"Kolon_L7_trust_01" + '
  '0.083*"12592" + 0.083*"fd=\'39" + 0.083*"occur" + 0.083*"write" + '
  '0.083*"error=\'Connection" + 0.083*"refuse" + 0.083*"error"'),
 (2,
  '0.109*"idle" + 0.109*"POLLERR" + 0.109*"occur" + 0.109*"fd=\'39" + '
  '0.109*"syslog-ng" + 0.109*"syslog" + 0.109*"Kolon_L7_trust_01" + '
  '0.109*"12592" + 0.001*"18:51:11" + 0.001*"18:46:10"'),
 (3,
  '0.123*"syslog" + 0.123*"syslog-ng" + 0.123*"Kolon_L7_trust_01" + '
  '0.123*"12592" + 0.122*"time_reopen=\'60" + 0.122*"Connection" + '
  '0.122*"break" + 0.001*"18:55:11" + 0.000*"18:51:11" + 0.000*"18:49:10"'),
 (4,
  '0.023*"iken" + 0.014*"members" + 0.014*"available" + '
  '0.014*"/Common/p_media_9005" + 0.014*"local0" + 0.008*"01010221:3" + '
  '0.008*"Pool" + 0.007*"19367" + 0.007*"19374" + 0.007*"tmm1"'),
 (5,
  '0.035*"tunnel-id=" + 0.035*"type=" + 0.035*"interface-name=" + '
  '0.035*"junos" + 0.035*"sequence-number=" + 0.035*"source-address=" + '
  '0.035*"destination-address=" + 0.035*"length=" + 0.035*"index=" + '
  '0.035*"203.225.253.71"'),
 (6,
  '0.080*"occur" + 0.080*"fd=\'39" + 0.080*"write" + 0.080*"error" + '
  '0.080*"refuse" + 0.080*"error=\'Connection" + 0.079*"12499" + '
  '0.079*"Kolon_L7_trust_02" + 0.079*"syslog-ng" + 0.079*"syslog"'),
 (7,
  '0.109*"idle" + 0.109*"POLLERR" + 0.109*"fd=\'39" + 0.109*"occur" + '
  '0.109*"12499" + 0.109*"Kolon_L7_trust_02" + 0.109*"syslog-ng" + '
  '0.109*"syslog" + 0.000*"18:43:06" + 0.000*"18:46:06"'),
 (8,
  '0.123*"syslog" + 0.123*"syslog-ng" + 0.123*"Kolon_L7_trust_02" + '
  '0.123*"12499" + 0.122*"Connection" + 0.122*"break" + '
  '0.122*"time_reopen=\'60" + 0.001*"19:20:06" + 0.000*"18:44:06" + '
  '0.000*"18:46:06"'),
 (9,
  '0.027*"stats" + 0.027*":514" + 0.027*"Duplicate" + 0.027*"counter" + '
  '0.027*"counter=\'udp" + 0.027*"null" + 0.022*"04:02:04" + 0.005*"11947" + '
  '0.005*"11994" + 0.003*"08:54:06"')]

In [181]:
result_df_1 = pd.DataFrame([i[1].split(' + ') for i in _1st])

In [182]:
for i in result_df_1:
  result_df_1['{}_rate'.format(i)]=result_df_1[i].str[:5]
  result_df_1[i]= result_df_1[i].str[6:]

In [183]:
temp = []
for i in range(10):
  temp += ['{}_rate'.format(i), i]

In [184]:
result_df_1 = result_df_1[temp]

결과 확인

In [190]:
result_df_1

,0_rate,0,1_rate,1,2_rate,2,3_rate,3,4_rate,4,5_rate,5,6_rate,6,7_rate,7,8_rate,8,9_rate,9
0,0.022,"""BMW_Opo_AS""",0.022,"""ge-1/0/0.0""",0.022,"""192.168.0.37""",0.010,"""779b9feb""",0.010,"""2021-01-06T18:39:28.539+09:00""",0.010,"""e7e5""",0.003,"""08:15:06""",0.003,"""03:32:08""",0.003,"""03:40:08""",0.003,"""03:47:08"""
1,0.084,"""syslog-ng""",0.084,"""syslog""",0.083,"""Kolon_L7_trust_01""",0.083,"""12592""",0.083,"""fd='39""",0.083,"""occur""",0.083,"""write""",0.083,"""error='Connection""",0.083,"""refuse""",0.083,"""error"""
2,0.109,"""idle""",0.109,"""POLLERR""",0.109,"""occur""",0.109,"""fd='39""",0.109,"""syslog-ng""",0.109,"""syslog""",0.109,"""Kolon_L7_trust_01""",0.109,"""12592""",0.001,"""18:51:11""",0.001,"""18:46:10"""
3,0.123,"""syslog""",0.123,"""syslog-ng""",0.123,"""Kolon_L7_trust_01""",0.123,"""12592""",0.122,"""time_reopen='60""",0.122,"""Connection""",0.122,"""break""",0.001,"""18:55:11""",0.000,"""18:51:11""",0.000,"""18:49:10"""
4,0.023,"""iken""",0.014,"""members""",0.014,"""available""",0.014,"""/Common/p_media_9005""",0.014,"""local0""",0.008,"""01010221:3""",0.008,"""Pool""",0.007,"""19367""",0.007,"""19374""",0.007,"""tmm1"""
5,0.035,"""tunnel-id=""",0.035,"""type=""",0.035,"""interface-name=""",0.035,"""junos""",0.035,"""sequence-number=""",0.035,"""source-address=""",0.035,"""destination-address=""",0.035,"""length=""",0.035,"""index=""",0.035,"""203.225.253.71"""
6,0.080,"""occur""",0.080,"""fd='39""",0.080,"""write""",0.080,"""error""",0.080,"""refuse""",0.080,"""error='Connection""",0.079,"""12499""",0.079,"""Kolon_L7_trust_02""",0.079,"""syslog-ng""",0.079,"""syslog"""
7,0.109,"""idle""",0.109,"""POLLERR""",0.109,"""fd='39""",0.109,"""occur""",0.109,"""12499""",0.109,"""Kolon_L7_trust_02""",0.109,"""syslog-ng""",0.109,"""syslog""",0.000,"""18:43:06""",0.000,"""18:46:06"""
8,0.123,"""syslog""",0.123,"""syslog-ng""",0.123,"""Kolon_L7_trust_02""",0.123,"""12499""",0.122,"""Connection""",0.122,"""break""",0.122,"""time_reopen='60""",0.001,"""19:20:06""",0.000,"""18:44:06""",0.000,"""18:46:06"""
9,0.027,"""stats""",0.027,""":514""",0.027,"""Duplicate""",0.027,"""counter""",0.027,"""counter='udp""",0.027,"""null""",0.022,"""04:02:04""",0.005,"""11947""",0.005,"""11994""",0.003,"""08:54:06"""


In [191]:
result_df_1.to_csv('/content/syslog_lda.csv')

## 이벤트 로그 1차

In [198]:
df = pd.read_excel('./data/이벤트 목록_log1.xls')
add = pd.read_excel('./data/이벤트 목록_log2.xls')
add2  = pd.read_excel('./data/이벤트 목록_log3.xls')
df = pd.concat([df,add, add2])

df.head()

,심각도,발생 일시,그룹경로,시스템명,호스트명,리소스 이름,대상,이벤트 타입,이벤트 소스,상세내용
0,INFO,2021-01-19 10:30:48,시스템그룹>HR,INF-INFPRJOBWS02( 172.30.171.22),infprjobws02,채용WAS21로그,시스템그룹>HR>INF-INFPRJOBWS02( 172.30.171.22)>moni...,logMessage,NaN,"BENIT_Client IP1 : 114.119.158.142, user_no : ..."
1,INFO,2021-01-19 10:30:48,시스템그룹>HR,INF-INFPRJOBWS02( 172.30.171.22),infprjobws02,채용WAS21로그,시스템그룹>HR>INF-INFPRJOBWS02( 172.30.171.22)>moni...,logMessage,NaN,"BENIT_Client IP2 : 114.119.158.142, user_no : ..."
2,INFO,2021-01-19 10:30:04,시스템그룹>HR,INF-INFPRJOBWS01(121.135.187.201),infprjobws01,채용WAS11로그,시스템그룹>HR>INF-INFPRJOBWS01(121.135.187.201)>mon...,logMessage,NaN,"BENIT_Client IP2 : 203.225.82.146, user_no : n..."
3,INFO,2021-01-19 10:30:04,시스템그룹>HR,INF-INFPRJOBWS01(121.135.187.201),infprjobws01,채용WAS11로그,시스템그룹>HR>INF-INFPRJOBWS01(121.135.187.201)>mon...,logMessage,NaN,"BENIT_Client IP2 : 203.225.82.146, user_no : n..."
4,INFO,2021-01-19 10:30:04,시스템그룹>HR,INF-INFPRJOBWS01(121.135.187.201),infprjobws01,채용WAS11로그,시스템그룹>HR>INF-INFPRJOBWS01(121.135.187.201)>mon...,logMessage,NaN,"BENIT_Client IP1 : 203.225.82.146, user_no : n..."


In [199]:
text = df[['상세내용']]
text.columns = ['Content']
text.head()

,Content
0,"BENIT_Client IP1 : 114.119.158.142, user_no : ..."
1,"BENIT_Client IP2 : 114.119.158.142, user_no : ..."
2,"BENIT_Client IP2 : 203.225.82.146, user_no : n..."
3,"BENIT_Client IP2 : 203.225.82.146, user_no : n..."
4,"BENIT_Client IP1 : 203.225.82.146, user_no : n..."


In [200]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [201]:
# 토큰화
text['Content'] = text.apply(lambda row: nltk.word_tokenize(row['Content']), axis=1)
text.head()

,Content
0,"[BENIT_Client, IP1, :, 114.119.158.142, ,, use..."
1,"[BENIT_Client, IP2, :, 114.119.158.142, ,, use..."
2,"[BENIT_Client, IP2, :, 203.225.82.146, ,, user..."
3,"[BENIT_Client, IP2, :, 203.225.82.146, ,, user..."
4,"[BENIT_Client, IP1, :, 203.225.82.146, ,, user..."


In [202]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [203]:
# 불용어 처리
stop = stopwords.words('english')
text['Content'] = text['Content'].apply(lambda x: [word for word in x if word not in (stop)])
text.head()

,Content
0,"[BENIT_Client, IP1, :, 114.119.158.142, ,, use..."
1,"[BENIT_Client, IP2, :, 114.119.158.142, ,, use..."
2,"[BENIT_Client, IP2, :, 203.225.82.146, ,, user..."
3,"[BENIT_Client, IP2, :, 203.225.82.146, ,, user..."
4,"[BENIT_Client, IP1, :, 203.225.82.146, ,, user..."


In [204]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [205]:
# 표제어 추출
from nltk.stem import WordNetLemmatizer
text['Content'] = text['Content'].apply(lambda x : [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])
text.head()

,Content
0,"[BENIT_Client, IP1, :, 114.119.158.142, ,, use..."
1,"[BENIT_Client, IP2, :, 114.119.158.142, ,, use..."
2,"[BENIT_Client, IP2, :, 203.225.82.146, ,, user..."
3,"[BENIT_Client, IP2, :, 203.225.82.146, ,, user..."
4,"[BENIT_Client, IP1, :, 203.225.82.146, ,, user..."


In [206]:
# 일정 길이 이하 단어 삭제 
tokenized_doc = text['Content'].apply(lambda x : [word for word in x if len(word) > 3])
tokenized_doc[:5]

0    [BENIT_Client, 114.119.158.142, user_no, null,...
1    [BENIT_Client, 114.119.158.142, user_no, null,...
2    [BENIT_Client, 203.225.82.146, user_no, null, ...
3    [BENIT_Client, 203.225.82.146, user_no, null, ...
4    [BENIT_Client, 203.225.82.146, user_no, null, ...
Name: Content, dtype: object

In [207]:
data = [i for i in tokenized_doc]

In [208]:
id2word = corpora.Dictionary(data)

In [209]:
corpus = [id2word.doc2bow(text) for text in data]

In [210]:
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]]


In [211]:
import warnings 
warnings.filterwarnings('ignore')

In [212]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                            id2word = id2word,
                                            num_topics = 10,
                                            random_state = 100,
                                            update_every =1,
                                            chunksize = 100,
                                            passes = 10,
                                            alpha= 'auto',
                                            per_word_topics = True
                                            )

In [213]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.076*"create_time" + 0.076*"session_id" + 0.076*"last_access_time" + '
  '0.076*"is_new" + 0.076*"1800" + 0.076*"interval_time" + 0.076*"user_no" + '
  '0.076*"BENIT_Client" + 0.053*"20201231" + 0.051*"null"'),
 (1,
  '0.032*"null" + 0.031*"like" + 0.030*"BENIT_Client" + 0.030*"user_no" + '
  '0.030*"application/xml" + 0.030*"Header" + 0.030*"Connection=Keep-Alive" + '
  '0.030*"X-WebLogic-KeepAliveSecs=30" + 0.030*"Info" + '
  '0.030*"Host=dream.kolon.com"'),
 (2,
  '0.092*"10.0" + 0.079*"Windows" + 0.079*"-1502104144" + 0.071*"Win64" + '
  '0.041*"image/avif" + 0.032*"SSCSID=MR21" + 0.030*"q=0.01" + '
  '0.030*"charset=UTF-8" + 0.030*"X-Requested-With=XMLHttpRequest" + '
  '0.022*"Sec-Fetch-Dest=empty"'),
 (3,
  '0.023*"iPhone" + 0.019*"Accept-Language=en-US" + '
  '0.016*"Accept-Language=ko-kr" + 0.016*"AppleWebKit/605.1.15" + '
  '0.011*"114.119.134.10" + 0.011*"Mobile/15E148" + 0.011*"12:14:23" + '
  '0.010*"27.113.11.232" + 0.010*"1609341263980" + '
  '0.010*"Fvm0NuBsqLQ

결과 정리

In [219]:
_1st = [(0,
  '0.076*"create_time" + 0.076*"session_id" + 0.076*"last_access_time" + '
  '0.076*"is_new" + 0.076*"1800" + 0.076*"interval_time" + 0.076*"user_no" + '
  '0.076*"BENIT_Client" + 0.053*"20201231" + 0.051*"null"'),
 (1,
  '0.032*"null" + 0.031*"like" + 0.030*"BENIT_Client" + 0.030*"user_no" + '
  '0.030*"application/xml" + 0.030*"Header" + 0.030*"Connection=Keep-Alive" + '
  '0.030*"X-WebLogic-KeepAliveSecs=30" + 0.030*"Info" + '
  '0.030*"Host=dream.kolon.com"'),
 (2,
  '0.092*"10.0" + 0.079*"Windows" + 0.079*"-1502104144" + 0.071*"Win64" + '
  '0.041*"image/avif" + 0.032*"SSCSID=MR21" + 0.030*"q=0.01" + '
  '0.030*"charset=UTF-8" + 0.030*"X-Requested-With=XMLHttpRequest" + '
  '0.022*"Sec-Fetch-Dest=empty"'),
 (3,
  '0.023*"iPhone" + 0.019*"Accept-Language=en-US" + '
  '0.016*"Accept-Language=ko-kr" + 0.016*"AppleWebKit/605.1.15" + '
  '0.011*"114.119.134.10" + 0.011*"Mobile/15E148" + 0.011*"12:14:23" + '
  '0.010*"27.113.11.232" + 0.010*"1609341263980" + '
  '0.010*"Fvm0NuBsqLQP0veRtzLa2yuagQ76fQLEGSqeG8S4imKZRxjKjla3"'),
 (4,
  '0.229*"1549373300" + 0.122*"20201230" + 0.013*"1609333810169" + '
  '0.013*"notionplus" + 0.013*"naver.com_20201230221138" + '
  '0.013*"EyKzxSP5oFuMof5cB-t3l3xQGBF2bxwqNqKLU2hvLuFCe2d8N1to" + '
  '0.013*"58.236.104.117" + 0.010*"221.141.150.118" + '
  '0.010*"1ea0CqnvBySEeL_bvbvjmFkJUyfpFbxuBXCzJO0REUZQgAZwKF89" + '
  '0.010*"1609338366447"'),
 (5,
  '0.072*"java.lang.NullPointerException" + '
  '0.034*"com.thinkm.mink.commons.exection.MinkException" + 0.034*"fail" + '
  '0.034*"AUTHENTICATE" + 0.027*"server" + 0.027*"mail" + 0.025*"Caused" + '
  '0.023*"failure" + 0.023*"connection" + 0.023*"-499"'),
 (6,
  '0.051*"User-Agent=Mozilla/5.0" + 0.048*"Gecko" + 0.048*"like" + '
  '0.048*"Trident/7.0" + 0.048*"rv:11.0" + 0.043*"Referer=https" + '
  '0.037*"Windows" + 0.030*"Accept-Language=ko" + 0.026*"image/jxr" + '
  '0.025*"Accept-Language=ko-KR"'),
 (7,
  '0.189*"20201230" + 0.123*"-41027595" + 0.031*"SSCSID=MR22" + '
  '0.020*"X-WebLogic-Force-JVMID=-41027595" + 0.010*"121.158.254.79" + '
  '0.009*"naver.com_20201230220554" + 0.009*"ges72229" + '
  '0.008*"vg20DvYyP6L1YztIbFkfQsHrxElQdr0jO-6siiwNIecXqNIcCy-q" + '
  '0.008*"1609338648114" + 0.008*"203.243.50.47"'),
 (8,
  '0.057*"Connection=Keep-Alive" + 0.057*"X-WebLogic-KeepAliveSecs=30" + '
  '0.057*"Info" + 0.057*"Header" + 0.057*"BENIT_Client" + 0.057*"user_no" + '
  '0.057*"Host=dream.kolon.com" + 0.057*"deflate" + '
  '0.057*"Accept-Encoding=gzip" + 0.052*"Accept=text/html"'),
 (9,
  '0.054*"q=0.9" + 0.038*"q=0.8" + 0.023*"like" + 0.023*"Referer=https" + '
  '0.022*"Gecko" + 0.022*"KHTML" + 0.022*"User-Agent=Mozilla/5.0" + '
  '0.022*"Safari/537.36" + 0.022*"AppleWebKit/537.36" + '
  '0.021*"Sec-Fetch-Site=same-origin"')]

In [220]:
result_df_1 = pd.DataFrame([i[1].split(' + ') for i in _1st])

In [221]:
for i in result_df_1:
  result_df_1['{}_rate'.format(i)]=result_df_1[i].str[:5]
  result_df_1[i]= result_df_1[i].str[6:]

In [222]:
temp = []
for i in range(10):
  temp += ['{}_rate'.format(i), i]

In [223]:
result_df_1 = result_df_1[temp]

결과 확인

In [224]:
result_df_1

,0_rate,0,1_rate,1,2_rate,2,3_rate,3,4_rate,4,5_rate,5,6_rate,6,7_rate,7,8_rate,8,9_rate,9
0,0.076,"""create_time""",0.076,"""session_id""",0.076,"""last_access_time""",0.076,"""is_new""",0.076,"""1800""",0.076,"""interval_time""",0.076,"""user_no""",0.076,"""BENIT_Client""",0.053,"""20201231""",0.051,"""null"""
1,0.032,"""null""",0.031,"""like""",0.030,"""BENIT_Client""",0.030,"""user_no""",0.030,"""application/xml""",0.030,"""Header""",0.030,"""Connection=Keep-Alive""",0.030,"""X-WebLogic-KeepAliveSecs=30""",0.030,"""Info""",0.030,"""Host=dream.kolon.com"""
2,0.092,"""10.0""",0.079,"""Windows""",0.079,"""-1502104144""",0.071,"""Win64""",0.041,"""image/avif""",0.032,"""SSCSID=MR21""",0.030,"""q=0.01""",0.030,"""charset=UTF-8""",0.030,"""X-Requested-With=XMLHttpRequest""",0.022,"""Sec-Fetch-Dest=empty"""
3,0.023,"""iPhone""",0.019,"""Accept-Language=en-US""",0.016,"""Accept-Language=ko-kr""",0.016,"""AppleWebKit/605.1.15""",0.011,"""114.119.134.10""",0.011,"""Mobile/15E148""",0.011,"""12:14:23""",0.010,"""27.113.11.232""",0.010,"""1609341263980""",0.010,"""Fvm0NuBsqLQP0veRtzLa2yuagQ76fQLEGSqeG8S4imKZR..."
4,0.229,"""1549373300""",0.122,"""20201230""",0.013,"""1609333810169""",0.013,"""notionplus""",0.013,"""naver.com_20201230221138""",0.013,"""EyKzxSP5oFuMof5cB-t3l3xQGBF2bxwqNqKLU2hvLuFCe...",0.013,"""58.236.104.117""",0.010,"""221.141.150.118""",0.010,"""1ea0CqnvBySEeL_bvbvjmFkJUyfpFbxuBXCzJO0REUZQg...",0.010,"""1609338366447"""
5,0.072,"""java.lang.NullPointerException""",0.034,"""com.thinkm.mink.commons.exection.MinkException""",0.034,"""fail""",0.034,"""AUTHENTICATE""",0.027,"""server""",0.027,"""mail""",0.025,"""Caused""",0.023,"""failure""",0.023,"""connection""",0.023,"""-499"""
6,0.051,"""User-Agent=Mozilla/5.0""",0.048,"""Gecko""",0.048,"""like""",0.048,"""Trident/7.0""",0.048,"""rv:11.0""",0.043,"""Referer=https""",0.037,"""Windows""",0.030,"""Accept-Language=ko""",0.026,"""image/jxr""",0.025,"""Accept-Language=ko-KR"""
7,0.189,"""20201230""",0.123,"""-41027595""",0.031,"""SSCSID=MR22""",0.020,"""X-WebLogic-Force-JVMID=-41027595""",0.010,"""121.158.254.79""",0.009,"""naver.com_20201230220554""",0.009,"""ges72229""",0.008,"""vg20DvYyP6L1YztIbFkfQsHrxElQdr0jO-6siiwNIecXq...",0.008,"""1609338648114""",0.008,"""203.243.50.47"""
8,0.057,"""Connection=Keep-Alive""",0.057,"""X-WebLogic-KeepAliveSecs=30""",0.057,"""Info""",0.057,"""Header""",0.057,"""BENIT_Client""",0.057,"""user_no""",0.057,"""Host=dream.kolon.com""",0.057,"""deflate""",0.057,"""Accept-Encoding=gzip""",0.052,"""Accept=text/html"""
9,0.054,"""q=0.9""",0.038,"""q=0.8""",0.023,"""like""",0.023,"""Referer=https""",0.022,"""Gecko""",0.022,"""KHTML""",0.022,"""User-Agent=Mozilla/5.0""",0.022,"""Safari/537.36""",0.022,"""AppleWebKit/537.36""",0.021,"""Sec-Fetch-Site=same-origin"""


In [225]:
result_df_1.to_csv('/content/이벤트로그_lda.csv')

## WAS 로그 1차

In [226]:
df = pd.read_excel('./data/이벤트 목록_waslog1.xls')
add = pd.read_excel('./data/이벤트 목록_waslog2.xls')
add2  = pd.read_excel('./data/이벤트 목록_waslog3.xls')
df = pd.concat([df,add, add2])

df.head()

,심각도,발생 일시,그룹경로,시스템명,호스트명,리소스 이름,대상,이벤트 타입,이벤트 소스,상세내용
0,ERROR,2020-12-31 10:30:45,시스템그룹>유통/환경/서비스,kasswas01,kasswas01,[상] [수입차영업지원]-WAS-TomEE-KASS11(203.225.59.171),시스템그룹>유통/환경/서비스>BNS-KASSWAS01(203.225.59.171)>...,WASExceptionLog,/bsl/fi/bllmng/smrtbllsalesbllmng/controller/S...,"warning slow sql, over 30000 ms"
1,ERROR,2020-12-31 10:30:24,시스템그룹>홈페이지,klnweb01,klnweb01,[하] [코오롱베니트 홈페이지]-WAS-Tomcat-KBN_WEB_USER11(2...,시스템그룹>홈페이지>INF-KLNWEB01(203.225.255.108)>klnwe...,WASExceptionLog,/common/11727518.mp4,org.apache.catalina.connector.ClientAbortExcep...
2,ERROR,2020-12-31 10:29:56,인프라그룹>보안>서버,KII-DOCUWAS1(172.30.153.12),KII-DOCUWAS1,[상] [문서집중화]-WAS-WebLogic-KIIDOC13(172.30.153.12),인프라그룹>보안>서버>KII-DOCUWAS1(172.30.153.12)>KII-DO...,WASExceptionLog,/ecm/Servlet/ContentDownload,java.net.ConnectException: Connection refused:...
3,ERROR,2020-12-31 10:29:46,시스템그룹>베니트,bpoap02,bpoap02,[중] [BPO]-WAS-Tomcat-BPO12(203.225.59.84),시스템그룹>베니트 >BNT-BPOAP02(203.225.59.84)>bpoap02>...,WASExceptionLog,/bsl/tx/salesexcalcmng/salesexcalcmng/controll...,"java.sql.SQLException: ORA-12899: ""BPOADM"".""AC..."
4,ERROR,2020-12-31 10:29:39,시스템그룹>베니트,bpoap02,bpoap02,[중] [BPO]-WAS-Tomcat-BPO12(203.225.59.84),시스템그룹>베니트 >BNT-BPOAP02(203.225.59.84)>bpoap02>...,WASExceptionLog,/bsl/tx/salesexcalcmng/salesexcalcmng/controll...,"java.sql.SQLException: ORA-12899: ""BPOADM"".""AC..."


In [227]:
text = df[['상세내용']]
text.columns = ['Content']
text.head()

,Content
0,"warning slow sql, over 30000 ms"
1,org.apache.catalina.connector.ClientAbortExcep...
2,java.net.ConnectException: Connection refused:...
3,"java.sql.SQLException: ORA-12899: ""BPOADM"".""AC..."
4,"java.sql.SQLException: ORA-12899: ""BPOADM"".""AC..."


In [228]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [229]:
# 토큰화
text['Content'] = text.apply(lambda row: nltk.word_tokenize(row['Content']), axis=1)
text.head()

,Content
0,"[warning, slow, sql, ,, over, 30000, ms]"
1,[org.apache.catalina.connector.ClientAbortExce...
2,"[java.net.ConnectException, :, Connection, ref..."
3,"[java.sql.SQLException, :, ORA-12899, :, ``, B..."
4,"[java.sql.SQLException, :, ORA-12899, :, ``, B..."


In [230]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [231]:
# 불용어 처리
stop = stopwords.words('english')
text['Content'] = text['Content'].apply(lambda x: [word for word in x if word not in (stop)])
text.head()

,Content
0,"[warning, slow, sql, ,, 30000, ms]"
1,[org.apache.catalina.connector.ClientAbortExce...
2,"[java.net.ConnectException, :, Connection, ref..."
3,"[java.sql.SQLException, :, ORA-12899, :, ``, B..."
4,"[java.sql.SQLException, :, ORA-12899, :, ``, B..."


In [232]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [233]:
# 표제어 추출
from nltk.stem import WordNetLemmatizer
text['Content'] = text['Content'].apply(lambda x : [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])
text.head()

,Content
0,"[warn, slow, sql, ,, 30000, ms]"
1,[org.apache.catalina.connector.ClientAbortExce...
2,"[java.net.ConnectException, :, Connection, ref..."
3,"[java.sql.SQLException, :, ORA-12899, :, ``, B..."
4,"[java.sql.SQLException, :, ORA-12899, :, ``, B..."


In [234]:
# 일정 길이 이하 단어 삭제 
tokenized_doc = text['Content'].apply(lambda x : [word for word in x if len(word) > 3])
tokenized_doc[:5]

0                                  [warn, slow, 30000]
1    [org.apache.catalina.connector.ClientAbortExce...
2    [java.net.ConnectException, Connection, refuse...
3    [java.sql.SQLException, ORA-12899, BPOADM, AC_...
4    [java.sql.SQLException, ORA-12899, BPOADM, AC_...
Name: Content, dtype: object

In [235]:
data = [i for i in tokenized_doc]

In [236]:
id2word = corpora.Dictionary(data)

In [237]:
corpus = [id2word.doc2bow(text) for text in data]

In [238]:
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1)]]


In [239]:
import warnings 
warnings.filterwarnings('ignore')

In [240]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                            id2word = id2word,
                                            num_topics = 10,
                                            random_state = 100,
                                            update_every =1,
                                            chunksize = 100,
                                            passes = 10,
                                            alpha= 'auto',
                                            per_word_topics = True
                                            )

In [241]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.240*"resultset" + 0.218*"10000" + 0.218*"warn" + 0.218*"many" + '
  '0.020*"time" + 0.008*"발생했습니다" + 0.008*"bigint" + 0.008*"변환하는" + '
  '0.007*"nvarchar을" + 0.004*"Read"'),
 (1,
  '0.150*"connect" + 0.147*"java.net.ConnectException" + 0.147*"Connection" + '
  '0.146*"refuse" + 0.132*"warn" + 0.132*"slow" + 0.132*"30000" + '
  '0.003*"ORA-00001" + 0.003*"위배됩니다" + 0.001*"KOLONTALKDB_APP.UC_DEVICE_PK"'),
 (2,
  '0.104*"invalid" + 0.103*"number" + '
  '0.075*"java.sql.SQLIntegrityConstraintViolationException" + 0.058*"없습니다" + '
  '0.047*"entry" + 0.047*"\'PRIMARY" + 0.047*"Duplicate" + 0.047*"read" + '
  '0.041*"\'3003" + 0.029*"java.sql.SQLNonTransientConnectionException"'),
 (3,
  '0.197*"java.lang.NullPointerException" + 0.073*"java.net.SocketException" + '
  '0.069*"Broken" + 0.069*"pipe" + 0.054*"ClientAbortException" + '
  '0.033*"java.sql.SQLDataException" + 0.033*"ORA-01841" + 0.033*"-4713" + '
  '0.033*"+4713" + 0.033*"지정해야"'),
 (4,
  '0.200*"send" + 0.199*"org.apache.c

결과 정리

In [242]:
_1st = [(0,
  '0.240*"resultset" + 0.218*"10000" + 0.218*"warn" + 0.218*"many" + '
  '0.020*"time" + 0.008*"발생했습니다" + 0.008*"bigint" + 0.008*"변환하는" + '
  '0.007*"nvarchar을" + 0.004*"Read"'),
 (1,
  '0.150*"connect" + 0.147*"java.net.ConnectException" + 0.147*"Connection" + '
  '0.146*"refuse" + 0.132*"warn" + 0.132*"slow" + 0.132*"30000" + '
  '0.003*"ORA-00001" + 0.003*"위배됩니다" + 0.001*"KOLONTALKDB_APP.UC_DEVICE_PK"'),
 (2,
  '0.104*"invalid" + 0.103*"number" + '
  '0.075*"java.sql.SQLIntegrityConstraintViolationException" + 0.058*"없습니다" + '
  '0.047*"entry" + 0.047*"\'PRIMARY" + 0.047*"Duplicate" + 0.047*"read" + '
  '0.041*"\'3003" + 0.029*"java.sql.SQLNonTransientConnectionException"'),
 (3,
  '0.197*"java.lang.NullPointerException" + 0.073*"java.net.SocketException" + '
  '0.069*"Broken" + 0.069*"pipe" + 0.054*"ClientAbortException" + '
  '0.033*"java.sql.SQLDataException" + 0.033*"ORA-01841" + 0.033*"-4713" + '
  '0.033*"+4713" + 0.033*"지정해야"'),
 (4,
  '0.200*"send" + 0.199*"org.apache.catalina.connector.ClientAbortException" + '
  '0.199*"Failed" + 0.199*"java.io.IOException" + 0.199*"message" + '
  '0.000*"81,575,436" + 0.000*"81,575,433" + 0.000*"client" + 0.000*"avoid" + '
  '0.000*"consider"'),
 (5,
  '0.142*"response" + 0.124*"java.lang.IllegalStateException" + 0.107*"call" + '
  '0.102*"getOutputStream" + 0.102*"already" + 0.048*"AddToMap" + 0.042*"fail" '
  '+ 0.037*"nest" + 0.037*"exception" + 0.034*"process"'),
 (6,
  '0.123*"java.sql.SQLSyntaxErrorException" + 0.053*"ORA-00905" + '
  '0.049*"empty" + 0.044*"javax.servlet.ServletException" + 0.032*"File" + '
  '0.031*"find" + 0.031*"WEB-INF" + 0.031*"view" + 0.031*"sale" + '
  '0.031*"vrView.jsp"'),
 (7,
  '0.250*"java.sql.SQLException" + 0.247*"존재하지" + 0.247*"않습니다" + '
  '0.247*"ORA-00942" + 0.000*"RPHONE3" + 0.000*"CM_SMS_SEND_M" + 0.000*"시퀀스가" '
  '+ 0.000*"ORA-02289" + 0.000*"일치하지" + 0.000*"문자열과"'),
 (8,
  '0.158*"javax.servlet.ServletException" + 0.097*"ORA-01722" + 0.054*"error" '
  '+ 0.036*"com.microsoft.sqlserver.jdbc.SQLServerException" + 0.031*"commit" '
  '+ 0.028*"occur" + 0.027*"create" + 0.025*"file" + 0.025*"호스트에" + '
  '0.025*"끊겼습니다"'),
 (9,
  '0.333*"org.apache.catalina.connector.ClientAbortException" + '
  '0.316*"java.io.IOException" + 0.186*"상대편에" + 0.112*"파이프가" + '
  '0.029*"java.net.SocketTimeoutException" + 0.001*"reset" + 0.001*"directory" '
  '+ 0.000*"02월\\월보" + 0.000*"\\Temp\\zipTemp\\09bc614e81fc3b4b\\2017년" + '
  '0.000*"3월.zip\\2017년"')]

In [243]:
result_df_1 = pd.DataFrame([i[1].split(' + ') for i in _1st])

In [244]:
for i in result_df_1:
  result_df_1['{}_rate'.format(i)]=result_df_1[i].str[:5]
  result_df_1[i]= result_df_1[i].str[6:]

In [245]:
temp = []
for i in range(10):
  temp += ['{}_rate'.format(i), i]

In [246]:
result_df_1 = result_df_1[temp]

결과 확인

In [247]:
result_df_1

,0_rate,0,1_rate,1,2_rate,2,3_rate,3,4_rate,4,5_rate,5,6_rate,6,7_rate,7,8_rate,8,9_rate,9
0,0.240,"""resultset""",0.218,"""10000""",0.218,"""warn""",0.218,"""many""",0.020,"""time""",0.008,"""발생했습니다""",0.008,"""bigint""",0.008,"""변환하는""",0.007,"""nvarchar을""",0.004,"""Read"""
1,0.150,"""connect""",0.147,"""java.net.ConnectException""",0.147,"""Connection""",0.146,"""refuse""",0.132,"""warn""",0.132,"""slow""",0.132,"""30000""",0.003,"""ORA-00001""",0.003,"""위배됩니다""",0.001,"""KOLONTALKDB_APP.UC_DEVICE_PK"""
2,0.104,"""invalid""",0.103,"""number""",0.075,"""java.sql.SQLIntegrityConstraintViolationExcep...",0.058,"""없습니다""",0.047,"""entry""",0.047,"""'PRIMARY""",0.047,"""Duplicate""",0.047,"""read""",0.041,"""'3003""",0.029,"""java.sql.SQLNonTransientConnectionException"""
3,0.197,"""java.lang.NullPointerException""",0.073,"""java.net.SocketException""",0.069,"""Broken""",0.069,"""pipe""",0.054,"""ClientAbortException""",0.033,"""java.sql.SQLDataException""",0.033,"""ORA-01841""",0.033,"""-4713""",0.033,"""+4713""",0.033,"""지정해야"""
4,0.200,"""send""",0.199,"""org.apache.catalina.connector.ClientAbortExce...",0.199,"""Failed""",0.199,"""java.io.IOException""",0.199,"""message""",0.000,"""81,575,436""",0.000,"""81,575,433""",0.000,"""client""",0.000,"""avoid""",0.000,"""consider"""
5,0.142,"""response""",0.124,"""java.lang.IllegalStateException""",0.107,"""call""",0.102,"""getOutputStream""",0.102,"""already""",0.048,"""AddToMap""",0.042,"""fail""",0.037,"""nest""",0.037,"""exception""",0.034,"""process"""
6,0.123,"""java.sql.SQLSyntaxErrorException""",0.053,"""ORA-00905""",0.049,"""empty""",0.044,"""javax.servlet.ServletException""",0.032,"""File""",0.031,"""find""",0.031,"""WEB-INF""",0.031,"""view""",0.031,"""sale""",0.031,"""vrView.jsp"""
7,0.250,"""java.sql.SQLException""",0.247,"""존재하지""",0.247,"""않습니다""",0.247,"""ORA-00942""",0.000,"""RPHONE3""",0.000,"""CM_SMS_SEND_M""",0.000,"""시퀀스가""",0.000,"""ORA-02289""",0.000,"""일치하지""",0.000,"""문자열과"""
8,0.158,"""javax.servlet.ServletException""",0.097,"""ORA-01722""",0.054,"""error""",0.036,"""com.microsoft.sqlserver.jdbc.SQLServerException""",0.031,"""commit""",0.028,"""occur""",0.027,"""create""",0.025,"""file""",0.025,"""호스트에""",0.025,"""끊겼습니다"""
9,0.333,"""org.apache.catalina.connector.ClientAbortExce...",0.316,"""java.io.IOException""",0.186,"""상대편에""",0.112,"""파이프가""",0.029,"""java.net.SocketTimeoutException""",0.001,"""reset""",0.001,"""directory""",0.000,"""02월\월보""",0.000,"""\Temp\zipTemp\09bc614e81fc3b4b\2017년""",0.000,"""3월.zip\2017년"""


In [248]:
result_df_1.to_csv('/content/이벤트로그_WAS_lda.csv')